# Plots by Sebastian
## Plotting genres part of movies pr. year

### Importing libraries

In [5]:
import matplotlib.pyplot as plt 
from data_gen import gen_df
import numpy as np
import ipywidgets as widgets

### Generating dataframe from data-file

In [3]:
filename = 'imdb.csv'

df = gen_df(filename)
df['year'] = df['year'].astype(int)

### Plotting genres as part of total movies at the year
These are generated as:
$$
\text{Genre share} = \frac{\text{Total movies in genre in year}}{\text{Total movies in year}}
$$

In [22]:
# Generate genre list
genre_list = list(df)[12:40]

In [24]:
def _plot_1(df,genre):
    
    df['count'] = 1
    df['movies_year'] = df.groupby('year')['count'].transform(lambda x: x.sum())  

    # Plot year sum of different genres
    for i in genre:
    #i = genre

        df[f'{i}_year'] = df.groupby('year')[i].transform(lambda x: x.sum())
        df[f'{i}_share'] = df[f'{i}_year']/df['movies_year']

        y_share = df.groupby('year')[f'{i}_share'].first()

        #ax = plt.subplot(2,2,j)
        y_share.plot(kind='line', sharex='col', sharey='row')

        plt.xlabel('year')
        plt.ylabel('share of movies')
        plt.title(i)
        
    #plt.show()
    
def plot_1(df):
    
    widgets.interact(_plot_1,
                    df = widgets.fixed(df),
                    genre = widgets.SelectMultiple(
                        options = genre_list,
                        description = 'Genres',
                        disabled = False))
    
plot_1(df)


interactive(children=(SelectMultiple(description='Genres', options=('Action', 'Adult', 'Adventure', 'Animation…